In [24]:
import tensorflow as tf

from tensorflow.keras.layers import LSTM

# from tensorflow.keras.models import Sequential

from tensorflow.keras.losses import MeanSquaredError
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [25]:
true_path = "C:\\Users\Daniel\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\PortTraining_1404_002_FINAL_startingAt24.csv"
#training_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\train1_port_new.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\StbdTesting_1404_002.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\test1_stbd_new.csv"
prediction_path = "C:\\Users\\Daniel\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\DL_PredictedOutputTestGood2Q.csv"

In [26]:
df_true=pd.read_csv(true_path)
#df_test=pd.read_csv(test_path)
df_prediction=pd.read_csv(prediction_path)


In [27]:

#df_predictedDOA = df_prediction.iloc[:,4]
#df_prediction
df_predictedDOA = df_prediction[["PredictedDOA"]]
df_predictedDOA

,PredictedDOA
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
5480113,56.220299
5480114,69.492996
5480115,74.339043
5480116,76.695541


In [28]:
df_reference = df_true[["DOA", "AngleUncertainty"]]
df_reference

,DOA,AngleUncertainty
0,59.664999,96
1,54.539999,254
2,65.679999,255
3,65.729999,255
4,64.039999,255
...,...,...
5480113,82.134998,206
5480114,80.939998,97
5480115,80.624998,32
5480116,80.924998,48


In [29]:
df_reference['AngleUncertainty'].max()

255

In [30]:
df_reference['AngleUncertainty'].mean()

91.57663539361744

In [31]:
df_reference['AngleUncertainty'].var()

5982.814100658273

In [32]:
df_comparison = df_predictedDOA.join(df_reference)
df_comparison

,PredictedDOA,DOA,AngleUncertainty
0,0.000000,59.664999,96
1,0.000000,54.539999,254
2,0.000000,65.679999,255
3,0.000000,65.729999,255
4,0.000000,64.039999,255
...,...,...,...
5480113,56.220299,82.134998,206
5480114,69.492996,80.939998,97
5480115,74.339043,80.624998,32
5480116,76.695541,80.924998,48


In [33]:
theShape = df_comparison.shape
numRows= theShape[0]

In [34]:
numCols = 1
#5504457 
df_results = pd.DataFrame(index=range(numRows),columns=range(numCols))
#df_results = pd.DataFrame({'PassFail'})
#df_comparison.insert()
df_results.columns = {"PassFail"}
df_results

,PassFail
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
5480113,NaN
5480114,NaN
5480115,NaN
5480116,NaN


In [35]:
df_comparison = df_comparison.join(df_results)
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail
0,0.000000,59.664999,96,NaN
1,0.000000,54.539999,254,NaN
2,0.000000,65.679999,255,NaN
3,0.000000,65.729999,255,NaN
4,0.000000,64.039999,255,NaN
...,...,...,...,...
5480113,56.220299,82.134998,206,NaN
5480114,69.492996,80.939998,97,NaN
5480115,74.339043,80.624998,32,NaN
5480116,76.695541,80.924998,48,NaN


In [36]:
df_comparison['Delta'] = df_comparison['PredictedDOA'] - df_comparison['DOA']
df_comparison['AbsDelta'] = df_comparison['Delta'].abs()
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail,Delta,AbsDelta
0,0.000000,59.664999,96,NaN,-59.664999,59.664999
1,0.000000,54.539999,254,NaN,-54.539999,54.539999
2,0.000000,65.679999,255,NaN,-65.679999,65.679999
3,0.000000,65.729999,255,NaN,-65.729999,65.729999
4,0.000000,64.039999,255,NaN,-64.039999,64.039999
...,...,...,...,...,...,...
5480113,56.220299,82.134998,206,NaN,-25.914699,25.914699
5480114,69.492996,80.939998,97,NaN,-11.447002,11.447002
5480115,74.339043,80.624998,32,NaN,-6.285956,6.285956
5480116,76.695541,80.924998,48,NaN,-4.229457,4.229457


In [37]:
# less than (0.02*AngleUncertainty)/2, since this is the 2sigma value, and we want all values to be under 1 sigma.
df_comparison['PassFail'] = df_comparison['AbsDelta'] <= (0.02*df_comparison['AngleUncertainty']/2)
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail,Delta,AbsDelta
0,0.000000,59.664999,96,False,-59.664999,59.664999
1,0.000000,54.539999,254,False,-54.539999,54.539999
2,0.000000,65.679999,255,False,-65.679999,65.679999
3,0.000000,65.729999,255,False,-65.729999,65.729999
4,0.000000,64.039999,255,False,-64.039999,64.039999
...,...,...,...,...,...,...
5480113,56.220299,82.134998,206,False,-25.914699,25.914699
5480114,69.492996,80.939998,97,False,-11.447002,11.447002
5480115,74.339043,80.624998,32,False,-6.285956,6.285956
5480116,76.695541,80.924998,48,False,-4.229457,4.229457


In [38]:
PassCount = df_comparison['PassFail'].sum()
PassCount

275407

In [39]:
FailCount = (~df_comparison['PassFail']).sum()
FailCount

5204711

In [40]:
TotalCount = PassCount+FailCount
TotalCount

5480118

In [41]:
PercentPass = 100*(PassCount/TotalCount)
PercentPass

5.025566967718579